DO all this code in modular coding here only for research

In [2]:
import os

In [1]:
%pwd

'd:\\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project\\research'

In [3]:
#os.chdir("..")

In [4]:
%pwd

'd:\\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project'

In [4]:
#os.chdir('d:\\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project')

step 3#:prepare thr entity

In [5]:
#create entity here like data_ingestion create another dataclass na dinside that assigned parameter from params.yaml
#frozen=true not want to add any other parameter

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
#same as define in previous entity

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        #what ever define in configuration mention it here

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

6.update thr component


In [8]:
#from tf download vgg16 model

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 

In [9]:
#define class prepareBseModel

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    #
    #this will download vgG16 from the tf #get input shape from params #
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

#also save this model after it is saved
        self.save_model(path=self.config.base_model_path, model=self.model)

   #this is another static model that is _prepare_full_model

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        #after that flatten the layer and then create a custom dense layer
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        #this is full model where input os model.input and output is prediction
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        #configure model for the training
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        #at end returning full model
        full_model.summary()
        return full_model
    
    #this full model also need to save in this call prepare base full modeland giving paramter and save it.
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
 #this is static function nad save model operation


7.Now need to update the pipeline 

In [10]:
# it is just step of ruing one function
#initialize cinfigmanger from that call prepare base model
#initialize preparebase mdel and by this call get base model and update base mdeol
#model has been dwnloaded and also save inside the artifacts
#so have 2 model vgg16 model and updated base model 

try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-06-06 11:49:28,330: INFO: common: yaml file: d:\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project\config\config.yaml loaded successfully]
[2024-06-06 11:49:28,341: INFO: common: yaml file: d:\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project\params.yaml loaded successfully]
[2024-06-06 11:49:28,343: INFO: common: created directory at: artifacts]
[2024-06-06 11:49:28,345: INFO: common: created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 48s 1us/step
[2024-06-06 11:50:19,732: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 blo